In [ ]:
import numpy as np
import os,pickle,time
import matplotlib.pyplot as plt
from skimage import io
from skimage import img_as_float
from multislice import prop,prop_utils

In [ ]:
def get_focal_spot(focal_plane_):
    x_,y_ = np.where(focal_plane_==np.max(focal_plane_))
    x_ = x_[0]
    y_ = y_[0]
    if (x_ > 100) & (y_>100):
        n = 100
    else : 
        n = 50
    return focal_plane_[x_-n:x_+n,y_-n:y_+n]

In [ ]:
def make_zp_from_rings(num_zones):
    os.chdir('/raid/home/sajid/ZP_simulations/var_width_thickness/4kev/rings')
    zp = np.zeros((15000,15000))
    for i in range(num_zones):
        if i%2 == 1 :
            ring_ = np.load('ring_'+str(i)+'.npy')
            ring_ = tuple((ring_[0],ring_[1]))
            zp[ring_] = 1
    return zp

In [ ]:
def tilt(i,zp,thickness,parameters,out_dir):
    zp = make_zp_from_rings(num_zones)
    zp_coords = parameters['zp_coords']    
    step_xy = parameters['step_xy']
    energy = parameters['energy(in eV)']
    wavel = parameters['wavelength in m']
    f = parameters['focal_length']
    L = step_xy*np.shape(zp)[0] 
    n = np.shape(zp)[0]         
    zp_thickness = thickness 
    os.chdir(out_dir)
    
    t1 = time.time()
    print('claclulating for tilt angle : ',i)      
    theta = (i)*(np.pi/180)
    slope = np.tan(theta)
    x = np.linspace(zp_coords[0]*1e-6,zp_coords[1]*1e-6,n)
    X,Y = np.meshgrid(x,x)
    z1 = 2*np.pi*(1/wavel)*slope*X
    wave_in = np.multiply(np.ones((n,n),dtype='complex64'),np.exp(1j*(z1)))
    
    zp_beta  = parameters['beta']*zp
    zp_delta = parameters['delta']*zp
    
    number_of_steps_zp =  prop_utils.number_of_steps(step_xy,wavel,zp_thickness)*2
    wave_focus = prop_utils.propogate_through_object(wave_in,zp_delta,zp_beta,zp_thickness,step_xy,wavel,L,
                                                     number_of_steps_zp,d1=0,d2=f, xray_object='zone Plate')
    focal_spot = get_focal_spot(np.abs(wave_focus))
    io.imsave('tilt_'+str(i)+'str'+'.tiff',img_as_float(focal_spot))
    t2 = time.time()
    print('tilt image number :',i,'time taken :',(t2-t1))

In [ ]:
num_zones = np.array([50,100,200,300,400,500,600])
thickness = np.array([0.5e-6,1e-6,2e-6,4e-6,8e-6,10e-6])

In [ ]:
inputs = np.linspace(-1,1,50)
parameters = pickle.load(open('parameters.pickle','rb'))

In [ ]:
for z in num_zones:
    for t in thickness :
        out_dir = '/raid/home/sajid/ZP_simulations/var_width_thickness/4kev/output'+ str('zones_'+str(z)+'thickness_'+'str(t)')
        os.mkdir(out_dir)
        for i in inputs :
            zp =  make_zp_from_rings(z)
            tilt(i,zp,t,parameters,out_dir)